In [1]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.guillaumerozier.fr
Mail : guillaume.rozier@telecomnancy.net

README:s
This file contains script that generate France maps and GIFs. 
Single images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.
I'm currently cleaning this file, please ask me is something is not clear enough!
Requirements: please see the imports below (use pip3 to install them).

"""

"\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.guillaumerozier.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:s\nThis file contains script that generate France maps and GIFs. \nSingle images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.\nI'm currently cleaning this file, please ask me is something is not clear enough!\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [2]:
import france_data_management as data
import pandas as pd
from tqdm import tqdm
import json
import plotly.express as px
from datetime import datetime
import imageio
import multiprocessing
import locale
import shutil
import os
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [3]:
# Import data from Santé publique France
_, _, _, _, _, _, _, df_incid, _ = data.import_data()
df_incid = df_incid[df_incid["cl_age90"] == 0]

with open('data/france/dep.geojson') as response:
    depa = json.load(response)

  0%|          | 0/4 [00:00<?, ?it/s]/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.

 75%|███████▌  | 3/4 [00:01<00:00,  2.44it/s]

       dep        jour        pop    P  cl_age90 departmentCode  \
119625  75  2020-05-13   198578.0    0         9             75   
119626  75  2020-05-13   208074.0    0        19             75   
119627  75  2020-05-13   386230.0    6        29             75   
119628  75  2020-05-13   332595.0    2        39             75   
119629  75  2020-05-13   277369.0    2        49             75   
...     ..         ...        ...  ...       ...            ...   
121215  75  2020-10-04   213755.0   16        69             75   
121216  75  2020-10-04   163119.0   14        79             75   
121217  75  2020-10-04    82607.0   11        89             75   
121218  75  2020-10-04    26637.0    1        90             75   
121219  75  2020-10-04  2148271.0  218         0             75   

       departmentName  regionCode     regionName     T   incidence  \
119625          Paris        11.0  Ile-de-France  1451         NaN   
119626          Paris        11.0  Ile-de-France  1451 

In [4]:
def build_map(data_df, img_folder, date_val, date_str = "date", dep_str = "departement", color_str = 'indic_synthese', legend_title="legend_title", title="title", subtitle=""):
    for date in date_val:
        data_df_temp = data_df[data_df[date_str] == date]
        
        if len(data_df_temp) > 0:
            fig = px.choropleth(geojson = depa, 
                                locations = data_df_temp[dep_str], 
                                featureidkey="properties.code",
                                color = data_df_temp[color_str],
                                scope='europe',
                                #labels={color_str:"Couleur"},
                                #color_discrete_sequence = ["green", "orange", "red"],
                                #labels={'red':"Couleur", 'orange':'bla', 'green':'lol'},
                                color_discrete_map = {"Risque Faible":"#DAF7A6", "Alerte":"#b8002a", "Alerte Renforcée":"#7c0030", "Alerte Maximale":"#460d37"},
                                category_orders = {color_str :["Risque Faible", "Alerte", "Alerte Renforcée", "Alerte Maximale"]}
                                      )
            date_title = datetime.strptime(date, '%Y-%m-%d').strftime('%d %B')
            date_now = datetime.now().strftime('%d %B')

            fig.update_geos(fitbounds="locations", visible=False)

            fig.update_layout(
                margin={"r":0,"t":0,"l":0,"b":0},
                title={
                    'text': title,
                    'y':0.98,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},

                titlefont = dict(
                    size=30),

                annotations = [
                    dict(
                        x=0.54,
                        y=0.03,
                        xref='paper',
                        yref='paper',
                        xanchor = 'center',
                        text='Source : Santé publique France. Auteur : @guillaumerozier.',
                        showarrow = False
                    ),

                    dict(
                        x=0.55,
                        y=0.94,
                        xref='paper',
                        yref='paper',
                        text= "{}<br>{} (données du {})".format(subtitle, date_now, date_title),
                        showarrow = False,
                        font=dict(
                            size=20
                                )
                    )]
                 ) 

            fig.update_geos(
                #center=dict(lon=-30, lat=-30),
                projection_rotation=dict(lon=12, lat=30, roll=8),
                #lataxis_range=[-50,20], lonaxis_range=[0, 200]
            )
            #fig.show()
            if date == dates_deconf[-1]:
                fig.write_image((img_folder+"/{}.jpeg").format("latest"), scale=2, width=1200, height=800)
            fig.write_image((img_folder+"/{}.jpeg").format(date), scale=2, width=1200, height=800)
        else:
            print("no data")

In [5]:
def build_gif(file_gif, imgs_folder, dates):
    i=0
    with imageio.get_writer(file_gif, mode='I', duration=0.3) as writer: 
        for date in tqdm(dates):
            try:
                print((imgs_folder+"/{}.jpeg").format(date))
                image = imageio.imread((imgs_folder+"/{}.jpeg").format(date))
                writer.append_data(image)
                i+=1
                if i==len(dates):
                    for k in range(8):
                        writer.append_data(image)
            except:
                print("no image for "+str(date))

In [6]:
dates_deconf = list(dict.fromkeys(list(df_incid["jour"].values)))

date = [dates_deconf[-1]]
build_map(df_incid.sort_values(by=['incidence']), "images/charts/france/dep-map-incid-cat", date_val=date, date_str = "jour", dep_str = "dep", color_str = 'incidence_color', legend_title="", title="Incidence", subtitle="Nombre de cas hebdomadaires pour 100 000 habitants")

In [7]:
#df_incid[df_incid["dep"]=="75"][-7:]

,dep,jour,pop,P,cl_age90,departmentCode,departmentName,regionCode,regionName,T,incidence,incidence_color
121153,75,2020-09-28,2148271.0,1082,0,75,Paris,11.0,Ile-de-France,8540,66.704806,Alerte
121164,75,2020-09-29,2148271.0,1025,0,75,Paris,11.0,Ile-de-France,8055,65.494530,Alerte
121175,75,2020-09-30,2148271.0,1152,0,75,Paris,11.0,Ile-de-France,8279,69.870142,Alerte
121186,75,2020-10-01,2148271.0,1213,0,75,Paris,11.0,Ile-de-France,8026,72.895831,Alerte
121197,75,2020-10-02,2148271.0,1215,0,75,Paris,11.0,Ile-de-France,8091,74.571597,Alerte
121208,75,2020-10-03,2148271.0,669,0,75,Paris,11.0,Ile-de-France,3907,41.382116,Risque Faible
121219,75,2020-10-04,2148271.0,218,0,75,Paris,11.0,Ile-de-France,1366,14.150915,Risque Faible


In [17]:
#df_incid.loc[df_incid["dep"] == "75"]["P"].rolling(window=7).sum()/df_incid.loc[df_incid["dep"] == "75"]["pop"]*100000

119635           NaN
119646           NaN
119657           NaN
119668           NaN
119679           NaN
             ...    
121175    267.284714
121186    278.735783
121197    291.583324
121208    303.034394
121219    306.013534
Length: 145, dtype: float64

In [7]:
build_gif("images/charts/france/incid-cat.gif", "images/charts/france/dep-map-incid-cat", dates_deconf[-30:])


  0%|          | 0/30 [00:00<?, ?it/s]

images/charts/france/dep-map-incid-cat/2020-09-05.jpeg



  3%|▎         | 1/30 [00:00<00:19,  1.45it/s]

images/charts/france/dep-map-incid-cat/2020-09-06.jpeg



  7%|▋         | 2/30 [00:01<00:19,  1.44it/s]

...


 93%|█████████▎| 28/30 [00:20<00:01,  1.47it/s]

images/charts/france/dep-map-incid-cat/2020-10-03.jpeg



 97%|█████████▋| 29/30 [00:21<00:00,  1.48it/s]

images/charts/france/dep-map-incid-cat/2020-10-04.jpeg



100%|██████████| 30/30 [00:25<00:00,  1.16it/s]
